In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/keluhan-aplikasi/dataset_train_gpt.csv
/kaggle/input/keluhan-pengguna-augmented/dataset_train_gpt_augmented.csv


In [2]:
pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.40
    Uninstalling SQLAlchemy-2.0.40:
      Successfully uninstalled SQLAlchemy-2.0.40
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset, load_dataset
import torch

# Load tokenizer dan model dari Hugging Face
model_name = "cahya/gpt2-small-indonesian-522M"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Pastikan padding token ada
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

# 1. Load dataset
dataset = load_dataset('csv', data_files='/kaggle/input/keluhan-pengguna-augmented/dataset_train_gpt_augmented.csv')['train']

# 2. Gabungkan kolom 'Topik Permasalahan' dan 'Rekomendasi' jadi satu kolom 'text'
dataset = dataset.map(lambda example: {
        "text": "Keluhan: " + example["Review"] + " Reply: " + example["Reply"]
})

# 3. Split 80-20
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
eval_dataset = dataset['test']

# 4. Tokenisasi
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=False)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=False)

# 5. Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Untuk causal LM seperti GPT2
)

# 6. Konfigurasi Training
training_args = TrainingArguments(
    output_dir="./result",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    save_strategy="epoch",
    logging_steps=10,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    report_to=[]
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# 8. Mulai training
trainer.train()

2025-06-09 15:43:38.617509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749483818.823662      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749483818.883624      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/894k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/452k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/tmp/ipykernel_19/3871697704.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,0.361700,0.628254
2,0.272900,0.481493
3,0.215300,0.470019
4,0.174600,0.418826
5,0.144500,0.400846
6,0.135900,0.397246
7,0.120800,0.384776
8,0.116200,0.379723
9,0.098300,0.371231
10,0.108700,0.371149


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=1130, training_loss=0.20965297317082904, metrics={'train_runtime': 258.1068, 'train_samples_per_second': 34.869, 'train_steps_per_second': 4.378, 'total_flos': 587907072000000.0, 'train_loss': 0.20965297317082904, 'epoch': 10.0})

In [4]:
import math
eval_results = trainer.evaluate()
print(f'Perplexity: {math.exp(eval_results["eval_loss"]):.2f}')

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Perplexity: 1.45


In [5]:
# Simpan model & tokenizer
model.save_pretrained("/kaggle/working/gpt2rev-aug10test")
tokenizer.save_pretrained("/kaggle/working/gpt2rev-aug10test")

('/kaggle/working/gpt2rev-aug10test/tokenizer_config.json',
 '/kaggle/working/gpt2rev-aug10test/special_tokens_map.json',
 '/kaggle/working/gpt2rev-aug10test/vocab.json',
 '/kaggle/working/gpt2rev-aug10test/merges.txt',
 '/kaggle/working/gpt2rev-aug10test/added_tokens.json')

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load model dan tokenizer dari path lokal
model_path = "/kaggle/working/gpt2rev-aug10test"

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Pastikan model ke device yang tepat
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Fungsi untuk generate teks
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        repetition_penalty=1.5,
        top_p=0.92,
        top_k=50,
        temperature=0.7,
        do_sample=True,
        early_stopping=True
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Contoh penggunaan
prompt = "gabisa verifikasi otp"
generated = generate_text(prompt, max_length=100)
print(generated)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


gabisa verifikasi otp ya kuno sekali Reply: Mohon maaf atas kendala tersebut. Silahkan coba Log Out dan batang kayu In kembali secara berkala, pastikan aplikasi sudah terupdate, & koneksi internet lancar. Informasi perpanjangan SIM & layanan pengaduan (WA & Email) terlampir pada akun instagram @DigitalKorlantas.

Keripik  adalah bentuk pembayaran tidak berfungsi yang dilaporkan melalui email ke info@digitalkorlini.id agar tak menjadi mislead


In [7]:
import shutil

# Lokasi folder model
folder_path = '/kaggle/working/gpt2rev-aug10test'

# Lokasi output zip
output_zip_path = '/kaggle/working/gpt2rev-aug10test.zip'

# Membuat zip
shutil.make_archive(base_name=output_zip_path.replace('.zip', ''), format='zip', root_dir=folder_path)

print(f"Model sudah di-zip di {output_zip_path}")

Model sudah di-zip di /kaggle/working/gpt2rev-aug10test.zip


In [8]:
import shutil
import os

# Path folder working
folder_path = '/kaggle/working/'

# Menghapus semua file dan folder dalam folder working
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Menghapus folder
        else:
            os.remove(file_path)  # Menghapus file
        print(f"File/Folder {filename} berhasil dihapus.")
    except Exception as e:
        print(f"Gagal menghapus {filename}: {e}")

File/Folder gpt2rev-aug10test.zip berhasil dihapus.
File/Folder __notebook__.ipynb berhasil dihapus.
File/Folder gpt2rev-aug10test berhasil dihapus.
File/Folder result berhasil dihapus.
